# **Programming a RAG System in BigQuery**

Installing libraries for Vertex AI SDK and pandas

In [34]:
!pip install -U google-cloud-aiplatform pandas --quiet

Import statements for model initialization, vertexAI initialization and pandas

In [35]:
from vertexai import init
from vertexai.preview.generative_models import GenerativeModel
import pandas as pd

Initializing Vertex AI

In [36]:
init(project="qwiklabs-gcp-00-1c0ebb19fb7c", location="global")


To see the schema for the csv file from storage bucket

In [37]:
gcs_uri = "gs://labs.roitraining.com/aurora-bay-faqs/aurora-bay-faqs.csv"
df = pd.read_csv(gcs_uri)
df.head()


,question,answer
0,When was Aurora Bay founded?,Aurora Bay was founded in 1901 by a group of f...
1,What is the population of Aurora Bay?,Aurora Bay has a population of approximately 3...
2,Where is the Aurora Bay Town Hall located?,The Town Hall is located at 100 Harbor View Ro...
3,Who is the current mayor of Aurora Bay?,"The current mayor is Linda Greenwood, elected ..."
4,What are the primary industries in Aurora Bay?,The primary industries include commercial fish...


In [38]:
df.dtypes

,0
question,object
answer,object


Importing bigquery package

In [39]:
from google.cloud import bigquery

Creating bigquery client

In [40]:
bq_client = bigquery.Client(project="qwiklabs-gcp-00-1c0ebb19fb7c")

I have already created a dataset named 'AuroraBayFAQs' in bigquery and created Vertex AI connection also with proper IAM roles to access bigquery contents

Here i have embedding model, loading data and generate embedding for the data loaded



In [41]:
embedding_and_dataloading_pipeline_sql = """
-- 1. Create Embedding Model
CREATE OR REPLACE MODEL `AuroraBayFAQs.Embeddings`
REMOTE WITH CONNECTION `us.embedding_conn_us`
OPTIONS (ENDPOINT = 'text-embedding-005');

-- 2. Load CSV from GCS
LOAD DATA OVERWRITE AuroraBayFAQs.aurora_bay (
    question STRING,
    answer STRING
)
FROM FILES (
    format = 'CSV',
    uris = ['gs://labs.roitraining.com/aurora-bay-faqs/aurora-bay-faqs.csv'],
    skip_leading_rows = 1
);

-- 3. Generate Embeddings
CREATE OR REPLACE TABLE `AuroraBayFAQs.aurora_bay_embedded` AS
SELECT *
FROM ML.GENERATE_EMBEDDING(
    MODEL `AuroraBayFAQs.Embeddings`,
    (
      SELECT
        CONCAT("Q: ", question, " A: ", answer) AS content,
        question,
        answer
      FROM
        `AuroraBayFAQs.aurora_bay`
    )
);
"""

# Run the SQL
bq_client.query(embedding_and_dataloading_pipeline_sql).result()


Checking whether the data was loaded

In [42]:
query = "SELECT * FROM `AuroraBayFAQs.aurora_bay` LIMIT 5"
df = bq_client.query(query).to_dataframe()

print(df.shape)
print(df.head())


(5, 2)
                                         question  \
0                    When was Aurora Bay founded?   
1           What is the population of Aurora Bay?   
2      Where is the Aurora Bay Town Hall located?   
3         Who is the current mayor of Aurora Bay?   
4  What are the primary industries in Aurora Bay?   

                                              answer  
0  Aurora Bay was founded in 1901 by a group of f...  
1  Aurora Bay has a population of approximately 3...  
2  The Town Hall is located at 100 Harbor View Ro...  
3  The current mayor is Linda Greenwood, elected ...  
4  The primary industries include commercial fish...  


Creating vector search query and that will have top 5 results in aurora_bay_result table

In [43]:
def query_vector_search_data(user_input):
    search_sql = f"""
    CREATE OR REPLACE TABLE `AuroraBayFAQs.aurora_bay_result` AS
    SELECT
        query.query,
        base.content
    FROM
        VECTOR_SEARCH(
            TABLE `AuroraBayFAQs.aurora_bay_embedded`,
            'ml_generate_embedding_result',
            (
                SELECT
                    ml_generate_embedding_result,
                    content AS query
                FROM
                    ML.GENERATE_EMBEDDING(
                        MODEL `AuroraBayFAQs.Embeddings`,
                        (SELECT '{user_input}' AS content)
                    )
            ),
            top_k => 5,
            options => '{{"fraction_lists_to_search": 1.0}}'
        );
    """
    bq_client.query(search_sql).result()


Function to get refined response from Gemini model by filtering the top 5 vector search results

In [44]:
def get_response_from_gemini():
    result_df = bq_client.query("SELECT * FROM `AuroraBayFAQs.aurora_bay_result`").to_dataframe()

    context = "\n\n".join(
        result_df.apply(lambda row: f"Q: {row['query']}\nA: {row['content']}", axis=1)
    )

    prompt = (
        "You are a helpful assistant answering FAQs for Aurora Bay.\n"
        "Based on the following context, answer the user's question clearly in professinal way:\n\n"
        "If the answer is not relevant, don't hallucinate — just say 'No answer'."
        f"{context}"
    )

    gemini_model = GenerativeModel("gemini-2.5-pro-preview-06-05")
    response = gemini_model.generate_content(prompt)
    return response.text.strip()

Creating Interactive Chat

In [46]:
while True:
    user_query = input("You: ")
    if user_query.lower() == "exit":
        print("Goodbye!")
        break

    try:
        query_vector_search_data(user_query)
        reply = get_response_from_gemini()
        print(f"Bot: {reply}\n")
    except Exception as e:
        print(f"Error: {e}\n")


You: primary industries in aurora bay
Bot: The primary industries include commercial fishing, tourism, and small-scale logging in the nearby forests.

You: leader of aurora bay
Bot: The current mayor is Linda Greenwood, elected in 2021 for a four-year term.

You: where is aurora bay located
Bot: No answer

You: when was aurora founded?
Bot: Aurora Bay was founded in 1901 by a group of fur traders who recognized the region’s strategic coastal location.

You: exit
Goodbye!
